###  Testing with SDK v2

In [2]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml import command, Input
from azure.identity import DefaultAzureCredential
import sys

# Enter details of your AML workspace
subscription_id = "840b5c5c-3f4a-459a-94fc-6bad2a969f9d"
resource_group = "ml"
workspace = "ws02ent"
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

sys.path.append("../") # go to parent dir

In [5]:
from src.ray_on_aml.core import Ray_On_AML
from azure.ai.ml import  Input

import ray

ray_on_aml =Ray_On_AML(ml_client=ml_client, compute_cluster ="ds11" )

# head_ip = ray_on_aml.getRay(num_node=2,pip_packages=["torch==1.13.0", "azureml-mlflow"])
# client = ray.init(f"ray://{head_ip}:10001")
ray_on_aml.getRay(ci_is_head=True, num_node=2,pip_packages=["torch==1.13.0", "azureml-mlflow"])

Using azureml_py310_sdkv2 for the master node


Usage stats collection will be enabled by default in the next release. See https://github.com/ray-project/ray/issues/20857 for more details.
Traceback (most recent call last):
  File "/anaconda/envs/azureml_py38/bin//ray", line 8, in <module>
    sys.exit(main())
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/scripts/scripts.py", line 2269, in main
    return cli()
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/click/core.py", line 1130, in __call__
    return self.main(*args, **kwargs)
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/click/core.py", line 1055, in main
    rv = self.invoke(ctx)
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/click/core.py", line 1657, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/click/core.py", line 1404, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/anaconda/envs/azureml_p

Waiting cluster to start and return head node ip


In [4]:
ray.cluster_resources()

{'node:10.0.0.9': 1.0,
 'node:10.0.0.10': 1.0,
 'memory': 18718774068.0,
 'CPU': 4.0,
 'object_store_memory': 8634907852.0}

In [ ]:
client.disconnect()

###  Testing with SDK v1

In [14]:
ray_on_aml.shutdown(end_all_runs=True)

In [1]:
from azureml.core import Workspace
import sys
sys.path.append("../") # go to parent dir
import importlib


In [7]:
from src.ray_on_aml.core import Ray_On_AML
import ray
ws = Workspace.from_config()
ray_on_aml =Ray_On_AML(ws=ws, compute_cluster ="ds11",maxnode=2)
ray_on_aml.getRay(num_node=2,pip_packages=["torch==1.13.0", "azureml-mlflow"], ci_is_head=True)

# head_ip = ray_on_aml.getRay(num_node=2,pip_packages=["torch==1.13.0", "azureml-mlflow"], ci_is_head=True)

# client = ray.init(f"ray://{head_ip}:10001")


NameError: name 'Workspace' is not defined

In [4]:
ray.shutdown()

In [5]:
ray.cluster_resources()

RaySystemError: System error: Ray has not been started yet. You can start Ray with 'ray.init()'.

In [6]:
ray_on_aml.shutdown()

In [4]:
import ray

# Load data.
dataset = ray.data.read_csv("https://azuremlexamples.blob.core.windows.net/datasets/iris.csv").repartition(4)

train_dataset, valid_dataset = dataset.train_test_split(test_size=0.3)

# Create a test dataset by dropping the target column.
test_dataset = valid_dataset.drop_columns(cols=["target"])
# Create a preprocessor to scale some columns.

import numpy as np

from ray.data.preprocessors import Concatenator, Chain, StandardScaler

# Create a preprocessor to scale some columns and concatenate the result.
# preprocessor = Chain(
#     StandardScaler(columns=["mean radius", "mean texture"]),
#     Concatenator(exclude=["target"], dtype=np.float32),
# )


2022-12-12 17:55:17,761	INFO worker.py:1342 -- Connecting to existing Ray cluster at address: 10.0.0.7:6379...
2022-12-12 17:55:17,772	INFO worker.py:1528 -- Connected to Ray cluster.


RayTaskError(ImportError): [36mray::_get_read_tasks()[39m (pid=36833, ip=10.0.0.7)
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/ray/data/read_api.py", line 1380, in _get_read_tasks
    reader = ds.create_reader(**kwargs)
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/ray/data/datasource/file_based_datasource.py", line 219, in create_reader
    return _FileBasedDatasourceReader(self, **kwargs)
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/ray/data/datasource/file_based_datasource.py", line 378, in __init__
    paths, self._filesystem = _resolve_paths_and_filesystem(paths, filesystem)
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/ray/data/datasource/file_based_datasource.py", line 666, in _resolve_paths_and_filesystem
    raise ImportError(
ImportError: Please install fsspec to read files from HTTP.

In [ ]:
pip install pyarrow==6.0.1

In [ ]:
import torch
import torch.nn as nn
from torch.nn.modules.utils import consume_prefix_in_state_dict_if_present

from ray import train
from ray.air import session
from ray.air.config import ScalingConfig
from ray.train.torch import TorchCheckpoint, TorchTrainer


def create_model(input_features):
    return nn.Sequential(
        nn.Linear(in_features=input_features, out_features=16),
        nn.ReLU(),
        nn.Linear(16, 16),
        nn.ReLU(),
        nn.Linear(16, 1),
        nn.Sigmoid(),
    )


def train_loop_per_worker(config):
    batch_size = config["batch_size"]
    lr = config["lr"]
    epochs = config["num_epochs"]
    num_features = config["num_features"]

    # Get the Ray Dataset shard for this data parallel worker,
    # and convert it to a PyTorch Dataset.
    train_data = train.get_dataset_shard("train")
    # Create model.
    model = create_model(num_features)
    model = train.torch.prepare_model(model)

    loss_fn = nn.BCELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)

    for cur_epoch in range(epochs):
        for batch in train_data.iter_torch_batches(
            batch_size=batch_size, dtypes=torch.float32
        ):
            # "concat_out" is the output column of the Concatenator.
            inputs, labels = batch["concat_out"], batch["target"]
            optimizer.zero_grad()
            predictions = model(inputs)
            train_loss = loss_fn(predictions, labels.unsqueeze(1))
            train_loss.backward()
            optimizer.step()
        loss = train_loss.item()
        session.report({"loss": loss}, checkpoint=TorchCheckpoint.from_model(model))


num_features = len(train_dataset.schema().names) - 1

trainer = TorchTrainer(
    train_loop_per_worker=train_loop_per_worker,
    train_loop_config={
        "batch_size": 128,
        "num_epochs": 20,
        "num_features": num_features,
        "lr": 0.001,
    },
    scaling_config=ScalingConfig(
        num_workers=2,  # Number of workers to use for data parallelism.
        use_gpu=False,
        trainer_resources={"CPU": 1},  # so that the example works on Colab.
    ),
    datasets={"train": train_dataset},
    preprocessor=preprocessor,
)
# Execute training.
result = trainer.fit()
print(f"Last result: {result.metrics}")
# Last result: {'loss': 0.6559339960416158, ...}

In [ ]:
pip install --upgrade ray

In [ ]:
ray_on_aml.shutdown()

In [ ]:
import ray
client = ray.init(f"ray://{head_ip}:10001")


In [ ]:
client

In [ ]:
ray.shutdown()

In [ ]:
client.disconnect()